### Задание 1 

Основным модулем для работы с регулярными выражениями в питоне является re. Но есть ещё активно развивающийся regex. Найдите одно любое преимущество regex и опишите его, сопроводив примером (текст, на котором вы демонстрируете отличие, должен быть русским). Пользоваться можно любыми ресурсами, начните поиск с ссылок в семинарской тетрадке.

#### Ответ:

Модуль re не поддерживает работe со вложенными множествами (nested sets) и операциями на множествами (set operations). В дальнейшем эта возможность может быть добавлена.

Модуль regex имеет эту функцию и позволяет работать со вложенными множествами и операциями над ними. 

В примере мы с помощью регулярного выражения находим все слова, но исключаем знаки препинания.


In [8]:
import re
result = re.findall('[[А-я]--{P}]+', 'Книга, которую я прочитал, была очень интересная. Здорово')
print(result)

[]


Модуль re не выводит ничего.

In [9]:
import regex
result = regex.findall('(?V1)[[А-я]--{P}]+', 'Книга, которую я прочитал, была очень интересная. Здорово!')
print(result)

['Книга', 'которую', 'я', 'прочитал', 'была', 'очень', 'интересная', 'Здорово']


Модуль regex справляется с задачей.

### Задание 2

Скопируйте текст этой статьи - http://gramota.ru/class/coach/punct/45_177

Приведите его к нижнему регистру, удалите все знаки пунктуации, токенизируйте (любым способом). После этого сделайте стемминг и найдите по несколько примеров на каждый из видов ошибок:

1) два разных слова ошибочно свелись к одинаковой основе

2) две словоформы одного слова после стемминга отличаются друг от друга

3) слово не изменилось после стемминга (слово должно быть русским и длиннее 4 символов)

In [30]:
import string
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import os, json
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('russian')
mystem = Mystem()
morph = MorphAnalyzer()

In [31]:
text = """Обстоятельство – второстепенный член предложения, который обозначает признак действия или другого признака. Обстоятельства поясняют сказуемые или другие члены предложения. При синтаксическом разборе предложения обстоятельства подчеркиваются пунктирной линией (тире, точка, тире). Обстоятельства необходимо выделять запятыми в трех случаях. Рассмотрим каждый из них по очереди.

Первый случай

Обстоятельства в предложениях могут быть выражены четырьмя частями речи: 

наречием, например: Дворник встает рано;
деепричастием или деепричастным оборотом, например: Мужики, увидев помещика, сняли шапки;
инфинитивом, например: Все отправились на улицу (зачем?) расчищать снег; 
существительным, например: Я приехал в столицу (зачем?) на экскурсию.
Кроме этого, обстоятельство может быть выражено цельным по смыслу выражением, например: Дождь шел две недели подряд.

Необходимо запомнить, что выделять запятыми нужно обстоятельства, которые выражены деепричастием или деепричастным оборотом. Сравните: Он сидел, листая журнал, и скучал и Он сидел на скамейке. В первом предложении обстоятельство листая журнал выделяется, так как оно выражено деепричастным оборотом, а во втором обстоятельство на скамейке не обособляется, так как оно выражено существительным с предлогом.

Второй и третий случаи

По значению обстоятельства делятся на следующие основные группы: 

обстоятельства места, отвечающие на вопросы ГДЕ? КУДА? ОТКУДА? Например: Мы въехали (куда?) в город; 
обстоятельства времени, отвечающие на вопросы КОГДА? С КАКИХ ПОР? ДО КАКИХ ПОР? КАК ДОЛГО? Например: Мы прождали их около двух часов;
обстоятельства причины, которые отвечают на вопросы ПОЧЕМУ? ОТЧЕГО? ПО КАКОЙ ПРИЧИНЕ? Например: От усталости я не мог говорить; 
обстоятельства цели, которые отвечают на вопросы ЗАЧЕМ? ДЛЯ ЧЕГО? С КАКОЙ ЦЕЛЬЮ? Например: В санатории все приготовлено для лечения отдыхающих;
обстоятельства образа действия и степени, отвечающие на вопросы КАК? КАКИМ ОБРАЗОМ? В КАКОЙ СТЕПЕНИ? Например: Я несколько призадумался или Отец ни на шаг не отпускал меня;
обстоятельства условия, которые отвечают на вопрос ПРИ КАКОМ УСЛОВИИ? Например: При старании можно добиться успеха; 
обстоятельства уступки, которые отвечают на вопрос НЕСМОТРЯ НА ЧТО? Например: На улице, несмотря на мороз, было многолюдно; 
обстоятельства сравнения, отвечающие на вопрос КАК? Например: Голова у нее острижена, как у мальчишки.
В классификации обстоятельств по значению одним из восьми видов являются обстоятельства сравнения: они отвечают на вопрос КАК? и начинается с союзов КАК, СЛОВНО или БУДТО. Например: У нее были длинные волосы, мягкие, как лен. В некоторых учебниках и справочных пособиях обстоятельства сравнения называют также сравнительными оборотами. Необходимо запомнить, что обстоятельства сравнения в предложениях выделяются запятыми.

Еще один вид обстоятельств, который необходимо выделять запятыми, это обстоятельства уступки. Такие обстоятельства отвечают на вопрос НЕСМОТРЯ НА ЧТО? и начинаются с предлога НЕСМОТРЯ НА (или, реже, ВОПРЕКИ). Например: На улицах, несмотря на яркое солнце, горели фонари.

Итак, следует запомнить три случая, когда обстоятельства необходимо выделять запятыми: 

если они выражены деепричастным оборотом, 
если они представляют собой сравнительный оборот, 
если они начинаются с предлога НЕСМОТРЯ НА.
Еще раз рассмотрите примеры. Сверкая быстро в вышине, кружились искры. (Лермонтов) Она вдруг скрылась, как птичка, вспугнутая из кустарника (Лермонтов). Несмотря на непредвиденные трудности, работа закончена в срок.

Это правило имеет несколько важных примечаний: 

От деепричастий следует отличать наречия СТОЯ, СИДЯ, ЛЕЖА, МОЛЧА. НЕХОТЯ, ШУТЯ, НЕ ГЛЯДЯ, ИГРАЯ. Они образовались благодаря переходу слов из разряда деепричастий в наречия. Обстоятельства, выраженные такими словами, не обособляются. Например: Он стоял молча. 

Не выделяются также обстоятельства, выраженные фразеологическими оборотами, например: Они работали засучив рукава или Верчусь целый день как белка в колесе. 

Кроме обстоятельств уступки, которые выделяются всегда, могут факультативно обособляться обстоятельства, выраженные существительными с производными предлогами БЛАГОДАРЯ, СОГЛАСНО, ВОПРЕКИ, ВВИДУ, ВСЛЕДСТВИЕ, например: Благодаря хорошей погоде, мы купались в реке все лето. Обычно такие обстоятельства обособляются, если они распространены и стоят перед сказуемым.
"""

In [32]:
def normalize(text):
    tokens = list(tokenize(text, lowercase=True))
    return tokens

In [33]:
tkns = normalize(text)
print(tkns[:10])

['обстоятельство', 'второстепенный', 'член', 'предложения', 'который', 'обозначает', 'признак', 'действия', 'или', 'другого']


In [36]:
sorted([(word, stemmer.stem(word)) for word in normalize(text)])[:10]

[('а', 'а'),
 ('белка', 'белк'),
 ('благодаря', 'благодар'),
 ('благодаря', 'благодар'),
 ('благодаря', 'благодар'),
 ('будто', 'будт'),
 ('были', 'был'),
 ('было', 'был'),
 ('быстро', 'быстр'),
 ('быть', 'быт')]

#### Ответы:

1) два разных слова ошибочно свелись к одинаковой основе: ('целый', 'цел') - правильная основа, ('целью', 'цел') - неправильная основа, должно быть 'цель'

2) две словоформы одного слова после стемминга отличаются друг от друга: ('случаи', 'случа'), ('случай', 'случа'), ('случая', 'случ') - должно быть 'случа'

3) слово не изменилось после стемминга (слово должно быть русским и длиннее 4 символов):  Слова 'дворник', 'вопрос', 'например' и др. не изменились после стемминга, но это их основы, поэтому ошибки здесь нет. Неверные варианты я не обнаружил.

### Задание 3 

Проанализируйте список стоп-слов из нлтк (для русского). Какие ещё слова вы бы туда добавили? (5 слов будет достаточно, не забудьте аргументировать ваш выбор)

In [40]:
stops = stopwords.words('russian')
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

#### Ответы:

В список стоп-слов я бы добавил следующие слова: 
- свой 
- который
- весь
- ничто
- очень

Данные слова входят в список самых частотных слов русского языка, построенный на основе «Национального корпуса русского языка» и не имеют большого значения. 

### Задание 4

Скопируйте текст этой статьи - https://zona.media/news/2019/09/11/zhukov-expertise

Предобработайте текст двумя способами:
1) Разбейте текст на предложения sent_tokenize из nltk, токенизируйте каждое предложение с помощью регулярного выражения (попробуйте разные и выберите лучшее), лемматизируйте токены с помощью pymorphy2

2) предобработайте текст с помощью mystem3 

Ответьте на вопросы:
1) Что лучше разбивает на предложения (mystem или sent_tokenize)?
2) Что лучше токенизирует (mystem или ваша регулярка)? Если mystem, то можно ли улучшить регулярное выражение так, чтобы качество было выше?
3) Что лучше для лемматизации mystem или pymorphy?

In [27]:
text = """«Новая газета» опубликовала сокращенный вариант заключения экспертов Александра Коршикова и Анны Осокиной, которые анализировали видео из блога студента Высшей школы экономики Егора Жукова, обвиняемого в призывах к экстремизму в интернете (часть 2 статьи 280 УК).

Эксперты нашли экстремизм в четырех видео на YouTube-канале Жукова. По их мнению, в видео «отчетливо прослеживается мотив политической ненависти или вражды к действующей власти».

В заключении говорится, что Жуков приписывает действующей власти «враждебные действия», называет ее врагами, источником зла и вреда. Кроме того, студент призывает с этой властью бороться.

«Автор описывает катастрофическое положение населения в России, как следствие целенаправленной политики действующей власти <...>. Основным идеалом, которого следует достичь, является "смена власти в России" для того, чтобы "Россия стала свободной"», — цитирует издание экспертов.

В одном из видео, в котором Жуков размышлял об итогах митинга 7 октября 2017 года, эксперты увидели призыв бороться любым способом, «что включает в себя и действия насильственного характера, в частности, насильственный захват власти, вооруженный мятеж».

Егор Жуков — фигурант «московского дела». Сначала его обвиняли в участии в массовых беспорядках (часть 2 статьи 212 УК). В суде говорили, что среди материалов есть видео, где Жуков показывает кому-то направо на акции 27 июля. Из-за этого студента 2 августа отправили в СИЗО.

Однако через месяц дело о массовых беспорядках против Жукова закрыли, ему предъявили новое обвинение в призывах к экстремизму. По ходатайстве следствие суд перевел студента под домашний арест."""

#### 1 способ

In [29]:
sentences = sent_tokenize(text)
print(sentences[:5])

['«Новая газета» опубликовала сокращенный вариант заключения экспертов Александра Коршикова и Анны Осокиной, которые анализировали видео из блога студента Высшей школы экономики Егора Жукова, обвиняемого в призывах к экстремизму в интернете (часть 2 статьи 280 УК).', 'Эксперты нашли экстремизм в четырех видео на YouTube-канале Жукова.', 'По их мнению, в видео «отчетливо прослеживается мотив политической ненависти или вражды к действующей власти».', 'В заключении говорится, что Жуков приписывает действующей власти «враждебные действия», называет ее врагами, источником зла и вреда.', 'Кроме того, студент призывает с этой властью бороться.']


#### Регулярное выражение 1

In [89]:
tokens1 = []
for sentence in sentences:
    token = re.findall('\w+', sentence)
    tokens1.append(token)
print(tokens1[:5])


[['Новая', 'газета', 'опубликовала', 'сокращенный', 'вариант', 'заключения', 'экспертов', 'Александра', 'Коршикова', 'и', 'Анны', 'Осокиной', 'которые', 'анализировали', 'видео', 'из', 'блога', 'студента', 'Высшей', 'школы', 'экономики', 'Егора', 'Жукова', 'обвиняемого', 'в', 'призывах', 'к', 'экстремизму', 'в', 'интернете', 'часть', '2', 'статьи', '280', 'УК'], ['Эксперты', 'нашли', 'экстремизм', 'в', 'четырех', 'видео', 'на', 'YouTube', 'канале', 'Жукова'], ['По', 'их', 'мнению', 'в', 'видео', 'отчетливо', 'прослеживается', 'мотив', 'политической', 'ненависти', 'или', 'вражды', 'к', 'действующей', 'власти'], ['В', 'заключении', 'говорится', 'что', 'Жуков', 'приписывает', 'действующей', 'власти', 'враждебные', 'действия', 'называет', 'ее', 'врагами', 'источником', 'зла', 'и', 'вреда'], ['Кроме', 'того', 'студент', 'призывает', 'с', 'этой', 'властью', 'бороться']]


#### Регулярное выражение 2

In [83]:
tokens2 = []
for sentence in sentences:
    token = re.findall('\w+|[^\w\s]+', sentence)
    tokens2.append(token)
print(tokens2[:5])

[['«', 'Новая', 'газета', '»', 'опубликовала', 'сокращенный', 'вариант', 'заключения', 'экспертов', 'Александра', 'Коршикова', 'и', 'Анны', 'Осокиной', ',', 'которые', 'анализировали', 'видео', 'из', 'блога', 'студента', 'Высшей', 'школы', 'экономики', 'Егора', 'Жукова', ',', 'обвиняемого', 'в', 'призывах', 'к', 'экстремизму', 'в', 'интернете', '(', 'часть', '2', 'статьи', '280', 'УК', ').'], ['Эксперты', 'нашли', 'экстремизм', 'в', 'четырех', 'видео', 'на', 'YouTube', '-', 'канале', 'Жукова', '.'], ['По', 'их', 'мнению', ',', 'в', 'видео', '«', 'отчетливо', 'прослеживается', 'мотив', 'политической', 'ненависти', 'или', 'вражды', 'к', 'действующей', 'власти', '».'], ['В', 'заключении', 'говорится', ',', 'что', 'Жуков', 'приписывает', 'действующей', 'власти', '«', 'враждебные', 'действия', '»,', 'называет', 'ее', 'врагами', ',', 'источником', 'зла', 'и', 'вреда', '.'], ['Кроме', 'того', ',', 'студент', 'призывает', 'с', 'этой', 'властью', 'бороться', '.']]


#### pymorphy2

In [151]:
lemmas = [] 
for sentence in tokens1:
    for word in sentence:
        lemma = morph.parse(word)
        lemmas.append(lemma)
        
for lemma in lemmas[:10]:
    print('Слово:', lemma[0][0], ' ', 'Лемма:', lemma[0][2])

Слово: новая   Лемма: новый
Слово: газета   Лемма: газета
Слово: опубликовала   Лемма: опубликовать
Слово: сокращённый   Лемма: сократить
Слово: вариант   Лемма: вариант
Слово: заключения   Лемма: заключение
Слово: экспертов   Лемма: эксперт
Слово: александра   Лемма: александр
Слово: коршикова   Лемма: коршикова
Слово: и   Лемма: и


#### 2 способ

#### Mystem

In [10]:
f = open('text.txt', 'w')
f.write(text)
f.close()

In [17]:
os.system(' ./mystem -iscd --format json text.txt text_parsed.txt')

1

In [23]:
text = [json.loads(line) for line in open('text_parsed.txt')]

In [154]:
words_analized = mystem.analyze(text)
print(words_analized[:10])

[{'text': '«'}, {'analysis': [{'lex': 'новый', 'wt': 1, 'gr': 'A=им,ед,полн,жен'}], 'text': 'Новая'}, {'text': ' '}, {'analysis': [{'lex': 'газета', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}], 'text': 'газета'}, {'text': '» '}, {'analysis': [{'lex': 'опубликовывать', 'wt': 1, 'gr': 'V=прош,ед,изъяв,жен,сов'}], 'text': 'опубликовала'}, {'text': ' '}, {'analysis': [{'lex': 'сокращать', 'wt': 0.9846237673, 'gr': 'V,пе=(прош,вин,ед,прич,полн,муж,сов,страд,неод|прош,им,ед,прич,полн,муж,сов,страд)'}], 'text': 'сокращенный'}, {'text': ' '}, {'analysis': [{'lex': 'вариант', 'wt': 0.9998747153, 'gr': 'S,муж,неод=(вин,ед|им,ед)'}], 'text': 'вариант'}]


In [165]:
for parse in words_analized[:20]:
        if parse.get('analysis'):
            print('Слово:', parse['text'], ' ', 'Лемма:', parse['analysis'][0]['lex'])

Слово: Новая   Лемма: новый
Слово: газета   Лемма: газета
Слово: опубликовала   Лемма: опубликовывать
Слово: сокращенный   Лемма: сокращать
Слово: вариант   Лемма: вариант
Слово: заключения   Лемма: заключение
Слово: экспертов   Лемма: эксперт
Слово: Александра   Лемма: александр
Слово: Коршикова   Лемма: коршиков
Слово: и   Лемма: и


#### Ответы:

#### 1) Что лучше разбивает на предложения (mystem или sent_tokenize)?

Мне больше понравился sent_tokenize, потому что он более удобен в использовании и токенизация получается быстрее. Большой разницы в разбивке на предложения я не заметил.

#### 2) Что лучше токенизирует (mystem или ваша регулярка)? Если mystem, то можно ли улучшить регулярное выражение так, чтобы качество было выше?

Я думаю, что в данном случае токенизация не отличается значительно. Первое простое регулярное выражение выполнило задачу хорошо. В обоих случаях знаки пунктуацию были отброшены.


#### 3) Что лучше для лемматизации mystem или pymorphy?

По моему мнению mystem подходит для лемматизации лучше. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию. К тому же он более информативен и выдает результаты на русском.